Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
%cd instacart_2017_05_01

In [0]:
!ls -lh *.csv

# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
##### YOUR CODE HERE #####

import pandas as pd
import numpy as np

aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
order_products__prior = pd.read_csv("order_products__prior.csv")
order_products__train = pd.read_csv("order_products__train.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")

In [0]:
pd.set_option('display.max_rows', 100)

In [0]:

print("# aisles.shape")
print(aisles.shape)
print("# aisles.head(10)")
print(aisles.head(10))
print("# aisles.tail(10)")
print(aisles.tail(10))
print("# aisles.isnull().sum().sort_values()")
print(aisles.isnull().sum().sort_values())
print("# aisles.describe()")
print(aisles.describe())

In [0]:


print("# departments.shape")
print(departments.shape)
print("# departments.head(10)")
print(departments.head(10))
print("# departments.tail(10)")
print(departments.tail(10))
print("# departments.isnull().sum().sort_values()")
print(departments.isnull().sum().sort_values())
print("# departments.describe()")
print(departments.describe())

In [0]:


print("# order_products__prior.shape")
print(order_products__prior.shape)
print("# order_products__prior.head(10)")
print(order_products__prior.head(10))
print("# order_products__prior.tail(10)")
print(order_products__prior.tail(10))
print("# order_products__prior.isnull().sum().sort_values()")
print(order_products__prior.isnull().sum().sort_values())
print("# order_products__prior.describe()")
print(order_products__prior.describe())

In [0]:


print("# order_products__train.shape")
print(order_products__train.shape)
print("# order_products__train.head(10)")
print(order_products__train.head(10))
print("# order_products__train.tail(10)")
print(order_products__train.tail(10))
print("# order_products__train.isnull().sum().sort_values()")
print(order_products__train.isnull().sum().sort_values())
print("# order_products__train.describe()")
print(order_products__train.describe())

In [0]:


print("# orders.shape")
print(orders.shape)
print("# orders.head(10)")
print(orders.head(10))
print("# orders.tail(10)")
print(orders.tail(10))
print("# orders.isnull().sum().sort_values()")
print(orders.isnull().sum().sort_values())
print("# orders.describe()")
print(orders.describe())

In [0]:


print("# products.shape")
print(products.shape)
print("# products.head(10)")
print(products.head(10))
print("# products.tail(10)")
print(products.tail(10))
print("# products.isnull().sum().sort_values()")
print(products.isnull().sum().sort_values())
print("# products.describe()")
print(products.describe())

In [0]:
op = pd.concat([order_products__prior, order_products__train])
op_sub = op.drop(['add_to_cart_order','reordered'],axis='columns')

print(op_sub.shape)
print(op_sub.head(5))

In [0]:
p_sub = products.drop(['aisle_id','department_id'],axis='columns')

print(p_sub.shape)
print(p_sub.head(5))

In [0]:
sub_merge1 = pd.merge(op_sub, p_sub, on='product_id', how='inner')
sub_merge1 = sub_merge1.drop(['order_id','product_id'],axis='columns')

print(sub_merge1.shape)
print(sub_merge1.head(5))

In [149]:
prod_sum = sub_merge1.groupby(['product_name']).size()
prod_sum.sort_values(inplace=True,ascending=False)
print(prod_sum[0:10]) 

product_name
Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
dtype: int64


## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [43]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [44]:
# table2 -> tidy

table2 = table2.reset_index()
table2


,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [45]:

tidy2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])
tidy2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [46]:
tidy2 = tidy2.rename(columns={
    'index': 'treatment', 
    'variable': 'name', 
    'value': 'value'
})
tidy2

,treatment,name,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [47]:
tidy2.treatment = tidy2.treatment.str.replace('treatment', '')
tidy2

,treatment,name,value
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


In [56]:
# tidy -> table2

wide = tidy2.pivot_table(index='name', columns='treatment', values='value')
wide

treatment,a,b
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


In [0]:
import seaborn as sns

flights = sns.load_dataset('flights')

In [0]:
print("# flights.shape")
print(flights.shape)
print("# flights.head(10)")
print(flights.head(10))
print("# flights.tail(10)")
print(flights.tail(10))
print("# flights.isnull().sum().sort_values()")
print(flights.isnull().sum().sort_values())
print("# flights.describe()")
print(flights.describe())

In [70]:
# Flights Pivot Table

wide2 = flights.pivot_table(index='year', columns='month')
wide2

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####